In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('/Users/I566184/bachelorarbeit_coding/mesogeos/csv/shap_map_csv/shap_map_transformer.csv')